In [0]:
%python
from pyspark.sql.functions import (
    col, avg, sum as _sum, count, when, min as _min,
    lag
)
from pyspark.sql.window import Window


race_results_silver        = spark.table("f1.f1_silver_race_results")
sprint_results_silver      = spark.table("f1.f1_silver_sprint_results")
lap_times_silver           = spark.table("f1.f1_silver_lap_times")
pit_stops_silver           = spark.table("f1.f1_silver_pit_stops")
qualifying_silver          = spark.table("f1.f1_silver_qualifying")
driver_standings_silver    = spark.table("f1.f1_silver_driver_standings")
constructor_standings_silver = spark.table("f1.f1_silver_constructor_standings")
seasons_silver             = spark.table("f1.f1_silver_seasons")

In [0]:
%python
driver_season_base = (
    race_results_silver
    .groupBy("season", "driverId", "driverName", "teamName",
             "driverNationality", "teamNationality")
    .agg(
        _sum(when(col("statusDescription") == "Finished", 1)
             .otherwise(0)).alias("finished_races"),
        count("*").alias("races_count"),
        _sum(col("points")).alias("total_points"),
        # Only consider grid > 0 for average (exclude pit-lane starts marked as 0 if needed)
        avg(when(col("grid") > 0, col("grid"))).alias("avg_grid_position"),
        avg(col("position")).alias("avg_finish_position"),   # NULLs (DNFs) ignored in avg
        _sum(when(col("position") == 1, 1).otherwise(0)).alias("wins"),
        _sum(when(col("position") <= 3, 1).otherwise(0)).alias("podiums"),
        _sum(when(col("statusDescription") != "Finished", 1).otherwise(0)).alias("dnf_count")
    )
    .withColumn("points_per_race", col("total_points") / col("races_count"))
    .withColumn("avg_grid_vs_finish", col("avg_grid_position") - col("avg_finish_position"))
)

In [0]:
%python
from pyspark.sql.functions import row_number

w_final_round = Window.partitionBy("season").orderBy(col("round").desc())
driver_standings_final = (
    driver_standings_silver
    .withColumn("round_rank", row_number().over(w_final_round))
    .filter(col("round_rank") == 1)  # final race in the season
    .drop("round_rank")
)

In [0]:
%python
from pyspark.sql.functions import max, col

driver_standings_final_sel = driver_standings_silver.alias("ds_final").join(
    driver_standings_silver.groupBy("season").agg(
        max("round").alias("round")
    ),
    on=["season", "round"],
    how="inner"
).select(
    col("season"),
    col("driverId"),
    col("driverName"),
    col("points").alias("final_champ_points"),
    col("position").alias("final_champ_position"),
    col("wins").alias("season_wins_recorded")
)

driver_standings_final_sel = driver_standings_final_sel.dropDuplicates(
    ["season", "driverId"]
)

In [0]:
%python
driver_season_stats = (
    driver_season_base.alias("base")
    .join(
        driver_standings_final_sel.alias("ds"),
        on=[col("base.season") == col("ds.season"),
            col("base.driverId") == col("ds.driverId")],
        how="left"
    )
    .select(
        col("base.season"),
        col("base.driverId"),
        col("base.driverName"),
        col("base.teamName"),
        col("base.driverNationality"),
        col("base.teamNationality"),
        col("base.races_count"),
        col("base.finished_races"),
        col("base.total_points"),
        col("base.points_per_race"),
        col("base.avg_grid_position"),
        col("base.avg_finish_position"),
        col("base.avg_grid_vs_finish"),
        col("base.wins"),
        col("base.podiums"),
        col("base.dnf_count"),
        col("ds.final_champ_points"),
        col("ds.final_champ_position"),
        col("ds.season_wins_recorded")
    )
)

# Fill missing final championship fields with 0 / null-safe defaults
driver_season_stats = driver_season_stats.fillna(
    {"final_champ_points": 0.0, "final_champ_position": 0, "season_wins_recorded": 0}
)

(
    driver_season_stats
    .write
    .mode("overwrite")
    .format("delta")
    .partitionBy("season")
    .saveAsTable("f1.f1_gold_driver_season_stats")
)


In [0]:
%python
from pyspark.sql.functions import (
    count,
    sum as _sum,
    when,
    col,
    max
)

constructor_season_stats = (
    race_results_silver
    .groupBy("season", "constructorId", "teamName", "teamNationality")
    .agg(
        count("*").alias("entries_count"),
        _sum(col("points")).alias("team_total_points"),
        _sum(when(col("position") == 1, 1).otherwise(0)).alias("wins"),
        _sum(when(col("position") <= 3, 1).otherwise(0)).alias("podiums"),
        _sum(when(col("statusDescription") != "Finished", 1).otherwise(0)).alias("dnf_count")
    )
)

final_rounds = (
    constructor_standings_silver
    .groupBy("season")
    .agg(max("round").alias("max_round"))
)

cons_final_standings = (
    constructor_standings_silver.alias("cs")
    .join(
        final_rounds.alias("fr"),
        (col("cs.season") == col("fr.season")) &
        (col("cs.round") == col("fr.max_round")),
        "inner"
    )
    .select(
        col("cs.season"),
        col("cs.constructorId"),
        col("cs.teamName"),
        col("cs.points").alias("final_cons_points"),
        col("cs.position").alias("final_cons_position"),
        col("cs.wins").alias("season_cons_wins_recorded")
    )
    .dropDuplicates(["season", "constructorId"])
)

constructor_season_stats = (
    constructor_season_stats.alias("base")
    .join(
        cons_final_standings.alias("cs"),
        (col("base.season") == col("cs.season")) &
        (col("base.constructorId") == col("cs.constructorId")),
        "left"
    )
    .select(
        col("base.season"),
        col("base.constructorId"),
        col("base.teamName"),
        col("base.teamNationality"),
        col("base.entries_count"),
        col("base.team_total_points"),
        col("base.wins"),
        col("base.podiums"),
        col("base.dnf_count"),
        col("cs.final_cons_points"),
        col("cs.final_cons_position"),
        col("cs.season_cons_wins_recorded")
    )
)

constructor_season_stats = constructor_season_stats.fillna(
    {
        "final_cons_points": 0.0,
        "final_cons_position": 0,
        "season_cons_wins_recorded": 0
    }
)

(
    constructor_season_stats
    .write
    .mode("overwrite")
    .format("delta")
    .partitionBy("season")
    .saveAsTable("f1.f1_gold_constructor_season_stats")
)

In [0]:
%python
lap_agg = (
    lap_times_silver
    .groupBy("season", "raceId", "driverId")
    .agg(
        count("*").alias("lap_count_recorded"),
        _min("lapTimeMs").alias("best_lap_ms"),
        avg("lapTimeMs").alias("avg_lap_ms")
    )
)


pit_agg = (
    pit_stops_silver
    .groupBy("season", "raceId", "driverId")
    .agg(
        count("*").alias("pit_stop_count"),
        avg("pitTimeMs").alias("avg_pit_stop_ms"),
        _sum("pitTimeMs").alias("total_pit_stop_ms")
    )
)


quali_agg = (
    qualifying_silver
    .groupBy("season", "raceId", "driverId")
    .agg(
        _min("position").alias("quali_best_position") 
    )
)


sprint_agg = (
    sprint_results_silver
    .groupBy("season", "raceId", "driverId")
    .agg(
        _min("grid").alias("sprint_grid"),
        _min("position").alias("sprint_finish_position"),
        _sum("points").alias("sprint_points")
    )
)


w_ds = Window.partitionBy("season", "driverId").orderBy("round")

driver_standings_enriched = (
    driver_standings_silver
    .withColumn("prev_champ_points", lag("points", 1).over(w_ds))
    .withColumn("prev_champ_position", lag("position", 1).over(w_ds))
)

In [0]:
%python
ds_agg = (
    driver_standings_enriched
    .select(
        "season", "raceId", "driverId",
        "points",
        "position",
        "prev_champ_points",
        "prev_champ_position"
    )
    .withColumnRenamed("points", "champ_points_after_race")
    .withColumnRenamed("position", "champ_position_after_race")
)


race_driver_features = (
    race_results_silver.alias("res")
    # Lap features
    .join(
        lap_agg.alias("lap"),
        on=["season", "raceId", "driverId"],
        how="left"
    )
    # Pit features
    .join(
        pit_agg.alias("pit"),
        on=["season", "raceId", "driverId"],
        how="left"
    )
    # Qualifying features
    .join(
        quali_agg.alias("q"),
        on=["season", "raceId", "driverId"],
        how="left"
    )
    # Sprint features
    .join(
        sprint_agg.alias("sp"),
        on=["season", "raceId", "driverId"],
        how="left"
    )
    # Championship trajectory
    .join(
        ds_agg.alias("ds"),
        on=["season", "raceId", "driverId"],
        how="left"
    )
    .select(
        col("res.season"),
        col("res.raceId"),
        col("res.round"),
        col("res.raceName"),
        col("res.circuitId"),
        col("res.circuitName"),
        col("res.country"),
        col("res.driverId"),
        col("res.driverName"),
        col("res.teamName"),
        col("res.driverNationality"),
        col("res.teamNationality"),
        col("res.grid"),
        col("q.quali_best_position"),
        col("res.position").alias("race_finish_position"),
        col("res.positionText"),
        col("res.positionOrder"),
        col("res.points").alias("race_points"),
        col("res.laps").alias("race_laps"),
        col("res.statusDescription"),
        # Lap features
        col("lap.lap_count_recorded"),
        col("lap.best_lap_ms"),
        col("lap.avg_lap_ms"),
        # Pit features
        col("pit.pit_stop_count"),
        col("pit.avg_pit_stop_ms"),
        col("pit.total_pit_stop_ms"),
        # Sprint features
        col("sp.sprint_grid"),
        col("sp.sprint_finish_position"),
        col("sp.sprint_points"),
        # Championship features
        col("ds.champ_points_after_race"),
        col("ds.champ_position_after_race"),
        col("ds.prev_champ_points"),
        col("ds.prev_champ_position")
    )
)

# Handle missing numeric values: if a driver had no pit stops or no laps,
# we replace nulls with 0 so that analytics/models are easier.
race_driver_features = race_driver_features.fillna({
    "lap_count_recorded": 0,
    "best_lap_ms": 0.0,
    "avg_lap_ms": 0.0,
    "pit_stop_count": 0,
    "avg_pit_stop_ms": 0.0,
    "total_pit_stop_ms": 0.0,
    "sprint_grid": 0,
    "sprint_finish_position": 0,
    "sprint_points": 0.0,
    "champ_points_after_race": 0.0,
    "champ_position_after_race": 0,
    "prev_champ_points": 0.0,
    "prev_champ_position": 0
})

(
    race_driver_features
    .write
    .mode("overwrite")
    .format("delta")
    .partitionBy("season")
    .saveAsTable("f1.f1_gold_race_driver_features")
)

print("Gold tables created.")